In [51]:
from bs4 import BeautifulSoup
import re
import urllib.request
import pandas as pd

def ExtractSoup(url):
	""" Scrapes urls for BS4 soup """

	url = re.sub(r'\s+', '', url)
	req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
	page = urllib.request.urlopen(req)
	soup = BeautifulSoup(page, "html.parser")
	return soup

df = pd.read_csv('arbitrators.csv')

In [52]:
def GetMetaData(url):
    soup = ExtractSoup(url)
    
    meta_data = {'nationality':soup.findAll("font", {"class": "noir_normal"})[2].text,
                 'Baradmiss':soup.findAll("font", {"class": "noir_normal"})[3].text,
                'Languages':soup.findAll("font", {"class": "noir_normal"})[1].text}
    
    return meta_data

In [53]:
df['meta_data'] = df['URL_CASE'].apply(lambda x: GetMetaData(x))

In [54]:
df = pd.concat([df.drop(['meta_data'], axis=1), df['meta_data'].apply(pd.Series)], axis=1)

In [55]:
copy = df

In [56]:
clean_columns = df.columns[8:]

In [57]:
def Cleaner(text):
    try:
        if type(text) == list:
            text = text[0]
    except:
        text = str(text)
        pass
    
    removers = ['\r','\n','[',']',';']

    for remove in removers:
        text = text.replace(remove,'')
    return text.strip()

In [58]:
for col in clean_columns:
    try:
        df[col] = df[col].apply(lambda x: Cleaner(x))
    except Exception as e: 
        print(e)
        print(col)

In [59]:
df['outcome'] = df['outcome'].apply(lambda x: x.strip())

KeyError: 'outcome'

In [ ]:
df.to_csv('arbitrators2.csv')